In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, urlencode
import json
import lxml
import streamlit as st

import matplotlib.pyplot as plt

import uuid
import os
import sys
from pathlib import Path

import fitz  # PyMuPDF, pdf파일 변환

import bcrypt

from google.cloud import bigquery
from google.oauth2 import service_account
import gspread
from gspread_dataframe import set_with_dataframe
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload

import warnings

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"
sheetskey_path = KEY_PATH + "mido-project-426906-41a4b6d0e3db.json"
midopluskey_path = KEY_PATH + "midoplus.json"

warnings.filterwarnings("ignore")

In [3]:
def get_service_key(servicekey_path):

    with open(servicekey_path) as f:
        data = json.load(f)
    return data

In [4]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [5]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [6]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [7]:
# 비밀번호 생성 및 해싱 함수
def generate_hashed_password(phone_number):
    
    # 전화번호의 뒷자리 4자리로 비밀번호 생성
    password = phone_number[-4:]
    
    # # 비밀번호 해싱
    # password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())
    return password

### 빅쿼리

In [8]:
shopping_df = get_dataframe_from_bigquery('DATA_WAREHOUSE', 'g2b_data', midopluskey_path)
shopping_df_fin = shopping_df.drop('collection_Date',axis=1)

shopping_prod_df = get_dataframe_from_bigquery('DATA_MARTS', 'g2b_prod_data', midopluskey_path)
shopping_prod_df_fin = shopping_prod_df.fillna('')

news_df = get_dataframe_from_bigquery('DATA_MARTS', 'news_data', midopluskey_path)
news_df_fin = news_df.drop('collection_Date',axis=1)

budget_df = get_dataframe_from_bigquery('DATA_MARTS', 'list_up_budget_data_final', midopluskey_path)
budget_df_final = budget_df.sort_values(['지역명','자치단체명']).reset_index(drop=True)

edu_budget_df_final = get_dataframe_from_bigquery('DATA_MARTS','listup_edu_budget_data_final',midopluskey_path)
edu_budget_df_final = edu_budget_df_final.sort_values(['시도','시군구']).reset_index(drop=True)

budget_link_df = get_dataframe_from_bigquery('RAW_DATA', 'budget_link', midopluskey_path)
budget_link_df = budget_link_df.drop('collection_Date',axis=1)

budget_add_df_final = get_dataframe_from_bigquery('DATA_MARTS','budget_add_df_final',midopluskey_path)
budget_add_df_final = budget_add_df_final.sort_values(['지역명','자치단체명']).reset_index(drop=True)

### 구글 스프레드 시트

In [9]:
# Google Sheets 클라이언트 생성
sheets_scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
                "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

sheets_creds = service_account.Credentials.from_service_account_file(midopluskey_path, scopes=sheets_scope)
gc = gspread.authorize(sheets_creds)

### 지자체 교육청 예산 현황

In [10]:
# 스프레드시트 ID (URL에서 확인 가능)
business_sheet_id = '166xdkZYI-SDNwdEiI6-Kt-p1wsTwFTAYAVfuCiFly0E' ## 미도플러스 사업현황

In [11]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = '지자체'
spreadsheet_business = gc.open_by_key(business_sheet_id)
worksheet_business = spreadsheet_business.worksheet(WORKSHEET_NAME)

# 기존 데이터 지우기
worksheet_business.clear()


# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_business.update([budget_df_final.columns.values.tolist()] + budget_df_final.values.tolist())

# # 구글 시트에서 데이터 읽기
# data = worksheet_business.get_all_records()
# pd.DataFrame(data)

{'spreadsheetId': '166xdkZYI-SDNwdEiI6-Kt-p1wsTwFTAYAVfuCiFly0E',
 'updatedRange': "'지자체'!A1:M2407",
 'updatedRows': 2407,
 'updatedColumns': 13,
 'updatedCells': 31291}

In [20]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = '추경'
spreadsheet_business = gc.open_by_key(business_sheet_id)
worksheet_business = spreadsheet_business.worksheet(WORKSHEET_NAME)

# 구글 시트에서 데이터 읽기
data = worksheet_business.get_all_records()
budget_add_df = pd.DataFrame(data)

# 뉴스 추경 데이터 추출
news_add_df = budget_add_df[budget_add_df['확인']=='News'].reset_index(drop=True)

# 뉴스 추경 통합
budget_add_df_final = pd.concat([news_add_df,budget_add_df_final], axis=0).reset_index(drop=True)

# 기존 데이터 지우기
worksheet_business.clear()

# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_business.update([budget_add_df_final.columns.values.tolist()] + budget_add_df_final.values.tolist())

# # 구글 시트에서 데이터 읽기
# data = worksheet_business.get_all_records()
# pd.DataFrame(data)

{'spreadsheetId': '166xdkZYI-SDNwdEiI6-Kt-p1wsTwFTAYAVfuCiFly0E',
 'updatedRange': "'추경'!A1:O463",
 'updatedRows': 463,
 'updatedColumns': 15,
 'updatedCells': 6945}

In [21]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = '지자체링크'
spreadsheet_business = gc.open_by_key(business_sheet_id)
worksheet_business = spreadsheet_business.worksheet(WORKSHEET_NAME)

# 기존 데이터 지우기
worksheet_business.clear()


# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_business.update([budget_link_df.columns.values.tolist()] + budget_link_df.values.tolist())

# # 구글 시트에서 데이터 읽기
# data = worksheet_business.get_all_records()
# pd.DataFrame(data)

{'spreadsheetId': '166xdkZYI-SDNwdEiI6-Kt-p1wsTwFTAYAVfuCiFly0E',
 'updatedRange': "'지자체링크'!A1:C244",
 'updatedRows': 244,
 'updatedColumns': 3,
 'updatedCells': 732}

In [22]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = '교육청'
spreadsheet_edu = gc.open_by_key(business_sheet_id)
worksheet_edu = spreadsheet_edu.worksheet(WORKSHEET_NAME)

# 기존 데이터 지우기
worksheet_edu.clear()


# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_edu.update([edu_budget_df_final.columns.values.tolist()] + edu_budget_df_final.values.tolist())

# # 구글 시트에서 데이터 읽기
# data = worksheet_edu.get_all_records()
# pd.DataFrame(data)

{'spreadsheetId': '166xdkZYI-SDNwdEiI6-Kt-p1wsTwFTAYAVfuCiFly0E',
 'updatedRange': "'교육청'!A1:H154",
 'updatedRows': 154,
 'updatedColumns': 8,
 'updatedCells': 1232}

In [23]:
# # 새로운 시트 생성
# budget_df = pd.DataFrame(data)
# new_sheet_name = '지자체백업'
# new_sheet = spreadsheet_business.add_worksheet(title=new_sheet_name, rows=len(budget_df), cols=len(budget_df.columns))

# set_with_dataframe(new_sheet, budget_df)

In [24]:
# # 데이터프레임을 기존 워크시트에 업로드
# budget_df = pd.DataFrame(data)
# set_with_dataframe(worksheet, budget_df)

### 종합쇼핑몰 납품 상세

In [25]:
# 스프레드시트 ID (URL에서 확인 가능)
shopping_sheet_id = '16vld1WTJwsrWPD_kFcUX1LJb-9WAtY2gnPFG5nGEqZ0' ## 미도플러스 종합쇼핑몰 현황
spreadsheet_shopping = gc.open_by_key(shopping_sheet_id)

#### 전국 기준

In [26]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = '납품현황'
worksheet_shopping = spreadsheet_shopping.worksheet(WORKSHEET_NAME)

# 기존 데이터 지우기
worksheet_shopping.clear()

# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_shopping.update([shopping_df_fin.columns.values.tolist()] + shopping_df_fin.values.tolist())

{'spreadsheetId': '16vld1WTJwsrWPD_kFcUX1LJb-9WAtY2gnPFG5nGEqZ0',
 'updatedRange': "'납품현황'!A1:AL14555",
 'updatedRows': 14555,
 'updatedColumns': 38,
 'updatedCells': 553090}

#### 미도플러스, 에코 기준

In [27]:
shopping_df_fin_mido = shopping_df_fin[shopping_df_fin['업체명'].str.contains('에코그라운드|미도플러스')].drop_duplicates().reset_index(drop=True)

In [28]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = '납품현황_미도'
worksheet_shopping = spreadsheet_shopping.worksheet(WORKSHEET_NAME)

# 기존 데이터 지우기
worksheet_shopping.clear()

# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_shopping.update([shopping_df_fin_mido.columns.values.tolist()] + shopping_df_fin_mido.values.tolist())

{'spreadsheetId': '16vld1WTJwsrWPD_kFcUX1LJb-9WAtY2gnPFG5nGEqZ0',
 'updatedRange': "'납품현황_미도'!A1:AL1119",
 'updatedRows': 1119,
 'updatedColumns': 38,
 'updatedCells': 42522}

### 종합쇼핑몰 품목 정보

In [29]:
# 스프레드시트 ID (URL에서 확인 가능)
shopping_sheet_id = '16vld1WTJwsrWPD_kFcUX1LJb-9WAtY2gnPFG5nGEqZ0' ## 미도플러스 종합쇼핑몰 현황
spreadsheet_shopping = gc.open_by_key(shopping_sheet_id)

In [30]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = '품목정보'
worksheet_shopping_prod = spreadsheet_shopping.worksheet(WORKSHEET_NAME)

# 기존 데이터 지우기
worksheet_shopping_prod.clear()

# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_shopping_prod.update([shopping_prod_df_fin.columns.values.tolist()] + shopping_prod_df_fin.values.tolist())

{'spreadsheetId': '16vld1WTJwsrWPD_kFcUX1LJb-9WAtY2gnPFG5nGEqZ0',
 'updatedRange': "'품목정보'!A1:AH614",
 'updatedRows': 614,
 'updatedColumns': 34,
 'updatedCells': 20876}

In [31]:
# # 데이터프레임을 기존 워크시트에 업로드
# budget_df = pd.DataFrame(data)
# set_with_dataframe(worksheet, budget_df)

### 뉴스스크랩

In [32]:
# 스프레드시트 ID (URL에서 확인 가능)
news_sheet_id = '1BnbDPv79Y44RpbMmZA2wFiFlM3mNN5xPtyOXFW0ABBk' ## midoplus news
spreadsheet_news = gc.open_by_key(news_sheet_id)

In [33]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = 'news'
worksheet_news = spreadsheet_news.worksheet(WORKSHEET_NAME)

# 기존 데이터 지우기
worksheet_news.clear()

# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_news.update([news_df_fin.columns.values.tolist()] + news_df_fin.values.tolist())

{'spreadsheetId': '1BnbDPv79Y44RpbMmZA2wFiFlM3mNN5xPtyOXFW0ABBk',
 'updatedRange': 'news!A1:D116',
 'updatedRows': 116,
 'updatedColumns': 4,
 'updatedCells': 464}

### 생산일지
- Cloud Functions 함수

#### 생산일지 업로드 로그

In [48]:
# 스프레드시트 ID (URL에서 확인 가능)
upload_sheet_id = '1sTpLxbmOdpGwSQDbvg02Eh1gxjVwSfI5PtMQ9sr-XmM' ## 계약관리, 생산일지 업로드
spreadsheet_upload = gc.open_by_key(upload_sheet_id)

In [49]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = 'upload_log'
worksheet_upload = spreadsheet_upload.worksheet(WORKSHEET_NAME)

# 구글 시트에서 데이터 읽기
data = worksheet_upload.get_all_records()
upload_log_df = pd.DataFrame(data)
upload_log_df

,업체명,File Upload,업로드날짜
0,미도플러스,upload_log_production_Files_/202408081ff00.Fil...,2024-09-04


In [50]:
# 폴더 내 파일리스트 확인
# 특정 폴더 ID
folder_id = '1JdzxjnumAqkWaVEDNPTjiqdvrtQCBuaK' ## 'upload_log_Files_' 폴더id

# Google Drive API 클라이언트 생성
drive_service = build('drive', 'v3', credentials=sheets_creds)

# 폴더 내의 모든 파일 리스트 가져오기
results = drive_service.files().list(
    q=f"'{folder_id}' in parents",
    spaces='drive',
    fields="nextPageToken, files(id, name, mimeType, createdTime, modifiedTime)",
    # pageSize=100  # 한 번에 최대 100개의 파일을 가져옵니다.
).execute()

items = results.get('files', [])

# if not items:
#     print('No files found.')
# else:
#     print(f"Files in folder '{folder_id}':")
#     for item in items:
#         print(f"File Name: {item['name']}, File ID: {item['id']}, Created Time: {item['createdTime']}, Modified Time: {item['modifiedTime']}")

upload_file_df = pd.DataFrame(items)
upload_file_df

,mimeType,id,name,createdTime,modifiedTime
0,application/vnd.openxmlformats-officedocument....,1iCFnpDS4tvtVhMz8X8Dga36ix00ROTRE,202408081ff00.File Upload.012200.xlsx,2024-09-04T01:22:00.823Z,2024-09-04T01:22:07.645Z


#### 업로드 생산일지 업데이트

In [51]:
# 최근 업로드된 파일 불러오기
recent_upload_nm = upload_log_df.iloc[-1]['File Upload'].split('/')[1]
file_id = upload_file_df[upload_file_df['name']==recent_upload_nm]['id'].iloc[0] ## 가장 최근 업로드한 파일 id

# 파일 다운로드
request = drive_service.files().get_media(fileId=file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False

while not done:
    status, done = downloader.next_chunk()
    print(f"Download {int(status.progress() * 100)}% complete.")

# 메모리에서 파일 읽기
fh.seek(0)
upload_df = pd.read_excel(fh,header=5) ## 업로드된 생산일지 원본

Download 100% complete.


In [30]:
# 업로드된 생산일지 전처리
# 불필요데이터 제거
upload_df_fin = upload_df.iloc[:upload_df['R/NO'].last_valid_index()+1]

# 모든 컬럼 NaN 제거
upload_df_fin = upload_df_fin.dropna(how='all')

# 컬럼형식 맞추기
upload_df_fin = upload_df_fin.rename(columns={'날짜':'생산일자', '일자':'생산일자'})
upload_df_fin = upload_df_fin.rename(columns={'제직량 M':'제직량', '입고량 M':'입고량'})

# 일자, 구장명 채우기
upload_df_fin['생산일자'] = upload_df_fin['생산일자'].ffill()
upload_df_fin['구장명'] = upload_df_fin['구장명'].ffill()

# '오더량'부터 'YARN'까지의 컬럼을 '구장명' 기준 첫 행으로 채우기
if "계약코드" in upload_df_fin.columns:
    columns_to_fill = ['계약코드','오더량', 'ITEM', 'SPI', 'PH', 'PW', 'T/F TYPE', '기포지', '제직폭', '본수', '입고폭', 'YARN', 'COLOR', '도전사','R/NO']
else:
    columns_to_fill = ['오더량', 'ITEM', 'SPI', 'PH', 'PW', 'T/F TYPE', '기포지', '제직폭', '본수', '입고폭', 'YARN', 'COLOR', '도전사','R/NO']
for col in columns_to_fill:
    upload_df_fin[col] = upload_df_fin.groupby(['생산일자','구장명'])[col].transform(lambda x: x.fillna(x.iloc[0]))

# 제직량, 입고량 없는거 제거
upload_df_fin = upload_df_fin[~upload_df_fin['제직량'].isnull()].drop_duplicates().reset_index(drop=True)

# 업체명 컬럼 추가
upload_df_fin['업체명'] = upload_log_df.iloc[-1]['업체명']

# 스프레드시트 ID (URL에서 확인 가능)
production_sheet_id = '1dl6GyEkzarfgcKSmT9hwYzR63MhcGDox0z-gCa0_7OI' ## midoplus production_report --> 생산일지통합
spreadsheet_production = gc.open_by_key(production_sheet_id)

# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = 'production_report'
worksheet_production = spreadsheet_production.worksheet(WORKSHEET_NAME)

# 구글 시트에서 데이터 읽기
data = worksheet_production.get_all_records()
production_df = pd.DataFrame(data)

# 기존 생산일지와 병합
production_df_fin = pd.concat([production_df,upload_df_fin],axis=0).reset_index(drop=True)
production_df_fin = production_df_fin.fillna('')

def convert_date_format(date_str):
    if isinstance(date_str, str):
        if len(date_str.split('.')) == 3:
            # 기존 형식이 '연도. 월. 일' 인 경우
            return pd.to_datetime(date_str, format='%Y.%m.%d').strftime('%Y-%m-%d')
        elif len(date_str.split('.')) == 2:
            # 새 형식이 '월.일' 인 경우, 연도를 올해 연도로 가정
            return pd.to_datetime(str(datetime.now().year) + '.' + date_str, format='%Y.%m.%d').strftime('%Y-%m-%d')
    return date_str  # 변환할 수 없는 경우 또는 비문자형 데이터인 경우 원본 반환

def safe_convert(x):
    try:
        # 공백이나 NaN 처리
        if pd.isna(x) or str(x).strip() == '':
            return ''
        # float 또는 int 값을 int로 변환 후 str로 변환
        return str(int(float(x)))
    except (ValueError, TypeError):
        return ''

# 행의 고유한 ID값 적용
def generate_unique_id():
    return str(uuid.uuid4())

# '생산일자' 컬럼에 변환 함수 적용
production_df_fin = production_df_fin.astype({'생산일자':str,'납품요구접수일자':str,'납품기한일자':str})
production_df_fin['생산일자'] = production_df_fin['생산일자'].str.split('~').str[0] 
production_df_fin['생산일자'] = production_df_fin['생산일자'].str.replace('00:00:00','').str.replace(' ','').str.replace('0:00','').str.replace('2024-05-078','2024-05-08').str.replace('3.2223','3.23')
production_df_fin['생산일자'] = production_df_fin['생산일자'].apply(convert_date_format)
production_df_fin['납품요구접수일자'] = production_df_fin['납품요구접수일자'].str.replace(' ','').str.replace('0:00','')
production_df_fin['납품요구접수일자'] = production_df_fin['납품요구접수일자'].apply(convert_date_format)
production_df_fin['납품기한일자'] = production_df_fin['납품기한일자'].str.replace(' ','').str.replace('0:00','')
production_df_fin['납품기한일자'] = production_df_fin['납품기한일자'].apply(convert_date_format)

# 구글시트 업데이트 위해 NaN값 대체
production_df_fin = production_df_fin.fillna('')

# 계약코드 대체
production_df_fin['계약코드'] = np.where(production_df_fin['계약코드']=='',production_df_fin['납품요구번호'],production_df_fin['계약코드']) ## 계약코드 없는것 --> 납품요구번호로 대체
production_df_fin['계약코드'] = production_df_fin['계약코드'].astype(str)
production_df_fin['계약코드'] = production_df_fin['계약코드'].str.replace('-','')
production_df_fin['계약코드'] = production_df_fin['계약코드'].str.replace(' ','')
production_df_fin['계약코드'] = production_df_fin['계약코드'].str.replace('미정','')
production_df_fin['계약코드'] = production_df_fin['계약코드'].str.replace('사급','')
production_df_fin['계약코드'] = production_df_fin['계약코드'].str.replace('無','')
production_df_fin['계약코드'] = production_df_fin['계약코드'].str.replace('계약서','')
# production_df_fin['계약코드'] =production_df_fin['계약코드'].apply(safe_convert)

# 제품 오입력 수정
production_df_fin['ITEM'] = production_df_fin['ITEM'].str.replace(',','')

# 납품요구건명 수정
production_df_fin['납품요구건명'] = production_df_fin['납품요구건명'].apply(lambda x: ' '.join(x.split('\n')[1].strip().split(' ')[1:]) if '\n' in x else x) ## 납품요구건명 오정보 수정

# 모든 행에 대해 새로운 고유 ID 할당
production_df_fin['ID'] = ''
production_df_fin['ID'] = production_df_fin['ID'].apply(lambda x: generate_unique_id())

# 계약코드 임의 설정
production_df_fin['계약코드'] = np.where(production_df_fin['계약코드']=='', production_df_fin['ID'].str.split('-').str[0], production_df_fin['계약코드'])
production_df_fin['계약코드'] = production_df_fin.groupby(['업체명','구장명'])['계약코드'].transform('first')

# # 최종 데이터셋 정리
# production_df_fin = production_df_fin[[production_df_fin.columns[-1]] + list(production_df_fin.columns[:-1])] ## 백업데이터에서 가져올때 정리되어있음

# 기존 데이터 지우기
worksheet_production.clear()

# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_production.update([production_df_fin.columns.values.tolist()] + production_df_fin.values.tolist())

OverflowError: cannot convert float infinity to integer

### 인증서
- pdf파일 --> image형식으로 변환

In [16]:
# PDF 파일들이 있는 디렉토리 경로
pdf_directory = 'C:/py_src/midoproject/data/documents/'

# PDF 파일 목록 가져오기
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

# 각 PDF 파일에 대해 이미지로 변환 및 저장
for pdf_file in pdf_files:
    # PDF 파일의 전체 경로
    pdf_file_path = os.path.join(pdf_directory, pdf_file)

    # PDF 파일명만 추출 (확장자 제거)
    pdf_filename = os.path.splitext(pdf_file)[0]

    # PDF 파일 열기
    pdf_document = fitz.open(pdf_file_path)

    # 각 페이지를 이미지로 변환 및 저장
    for page_number in range(len(pdf_document)):
        page = pdf_document.load_page(page_number)  # 페이지 로드
        image = page.get_pixmap()  # 페이지를 이미지로 변환

        # 이미지 파일 경로 생성
        image_filename = f'{pdf_filename}_page_{page_number + 1}.jpg'
        image_file_path = os.path.join(pdf_directory, image_filename)

        # 이미지 저장
        image.save(image_file_path)

    # PDF 문서 닫기
    pdf_document.close()

print("모든 PDF 파일이 이미지로 변환되었습니다.")


모든 PDF 파일이 이미지로 변환되었습니다.


### 창고재고관리

In [21]:
# 스프레드시트 ID (URL에서 확인 가능)
inventory_sheet_id = '1dTxyA-gk-akC2zvRElX4NRX7I8QRxil0yLJOKso3REs' ## midoplus news
spreadsheet_inventory = gc.open_by_key(inventory_sheet_id)

In [22]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = 'inventory_management'
worksheet_inventory = spreadsheet_inventory.worksheet(WORKSHEET_NAME)

# 구글 시트에서 데이터 읽기
data = worksheet_inventory.get_all_records()
inventory_df = pd.DataFrame(data)
inventory_df

,ID,구분,날짜,제품명,입고량,출고량
0,,일반용,2024-01-15,KL20,,50
1,,일반용,2024-02-24,KL28,,584
2,,일반용,2024-02-26,KL20,,350
3,,일반용,2024-03-26,KL20,,150
4,,일반용,2024-04-02,KL35,,100
...,...,...,...,...,...,...
214,,쇼핑몰,2024-09-20,CL10,,4
215,,쇼핑몰,2024-09-20,KL28,,0
216,,쇼핑몰,2024-09-24,KL20,,4
217,,쇼핑몰,2024-09-24,KL20,,0


In [23]:
# 행의 고유한 ID값 적용
def generate_unique_id():
    return str(uuid.uuid4())

# 모든 행에 대해 새로운 고유 ID 할당
inventory_df['ID'] = ''
inventory_df['ID'] = inventory_df['ID'].apply(lambda x: generate_unique_id())

inventory_df['ID'] = inventory_df['ID'].str.split('-').str[0]

In [24]:
# 기존 데이터 지우기
worksheet_inventory.clear()

# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_inventory.update([inventory_df.columns.values.tolist()] + inventory_df.values.tolist())

{'spreadsheetId': '1dTxyA-gk-akC2zvRElX4NRX7I8QRxil0yLJOKso3REs',
 'updatedRange': 'inventory_management!A1:F220',
 'updatedRows': 220,
 'updatedColumns': 6,
 'updatedCells': 1320}